In [13]:
import os
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import plotly.express as px

In [14]:
df = pd.read_csv('../input/paddy-disease-classification/train.csv')
df.head()

In [15]:
df_val = df['label'].value_counts()
x = df_val.index
y = df_val.values

flight_data = sns.load_dataset("flights")
plt.figure(figsize=(10, 6))

# fig = sns.barplot(x=x, y=y, order=x)
fig = sns.barplot(x=x, y=y )
plt.xticks(rotation=90, fontsize=14);
plt.yticks(fontsize=14);

In [16]:
df['k_fold'] = np.nan

n_folds = 6
reversed = False

for label in df.label.unique():
    
    folds = list(range(n_folds))
    if reversed: folds.reverse()
    
    kf = KFold(n_splits=n_folds, random_state=42, shuffle=True)
    
    label_idxs = df[df.label==label].index
    
    kf.get_n_splits(label_idxs)

    for _, valid_index in kf.split(label_idxs):

        actual_fold = folds.pop(0)
        df_index = label_idxs[valid_index]
        df.loc[df_index, 'k_fold'] = actual_fold
    reversed = not reversed
        

In [17]:
df.groupby(['label', 'k_fold']).count()

In [18]:

graphic = df.groupby(['label', 'k_fold']).size().reset_index()
graphic.columns = ['label', 'k_fold', 'count']
fig = px.bar(
    graphic, x="k_fold", y="count",
    color='label', barmode='group',
    height=400
)
fig.show()

In [19]:
df.to_csv('df_kfold.csv', index=False)